<a href="https://colab.research.google.com/github/stef1949/Synthetic-RNA-Seq-Raw-Counts-Generator/blob/main/Synthetic_RNA_Seq_Raw_Counts_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install cupy-cuda12x

In [ ]:
# GPU Accellerated workflow
#!pip install cupy-cuda12x

In [ ]:
#import cupy as cp
import numpy as np  # Still needed for some CPU operations (e.g., Pandas conversion)
import pandas as pd

# Synthetic Bulk RNA-Seq Dataset Generation

Script to generate synthetic bulk RNA-seq count data with batch effects for batch correction evaluation. An optimized version that vectorizes the count generation to improve performance.
    
## Parameters:
* n_genes (int): Number of genes to simulate.
* n_samples (int): Total number of samples (should be divisible by n_batches).
* n_batches (int): Number of batches.
* de_fraction (float): Fraction of genes that are differentially expressed in treatment samples.
* fold_change (float): Fold change applied to the mean of differentially expressed genes in treatment.
* dispersion (float): Dispersion parameter for the negative binomial distribution.
* batch_effect_sigma (float): Standard deviation for batch effects (log-normal sigma).
* seed (int): Random seed for reproducibility.
    
## Output:
* pd.DataFrame: Count matrix with gene IDs as rows and sample IDs as columns.
* pd.DataFrame: Sample metadata with condition and batch information.

### Contact
For more information https://github.com/stef1949 :3

In [ ]:
# CPU version

#!/usr/bin/env python3

def generate_synthetic_rnaseq_data(n_genes=10000, n_samples=12, n_batches=2, de_fraction=0.1,
                                   fold_change=2.0, dispersion=0.2, batch_effect_sigma=0.2, seed=123):
    """
    Generate synthetic RNA-seq count data with batch effects using vectorized operations.

    Parameters:
        n_genes (int): Number of genes to simulate.
        n_samples (int): Total number of samples (should be divisible by n_batches).
        n_batches (int): Number of batches.
        de_fraction (float): Fraction of genes that are differentially expressed in treatment samples.
        fold_change (float): Fold change applied to the mean of differentially expressed genes in treatment.
        dispersion (float): Dispersion parameter for the negative binomial distribution.
        batch_effect_sigma (float): Standard deviation for batch effects (log-normal sigma).
        seed (int): Random seed for reproducibility.

    Returns:
        pd.DataFrame: Count matrix with gene IDs as rows and sample IDs as columns.
        pd.DataFrame: Sample metadata with condition and batch information.
    """
    np.random.seed(seed)

    # Ensure that n_samples is divisible by n_batches
    if n_samples % n_batches != 0:
        raise ValueError("n_samples must be divisible by n_batches")

    samples_per_batch = n_samples // n_batches

    # Create sample metadata: for each batch, assign an equal number of control and treatment samples.
    conditions = []
    batches = []
    for b in range(n_batches):
        if samples_per_batch % 2 == 0:
            conds = ["control"] * (samples_per_batch // 2) + ["treatment"] * (samples_per_batch // 2)
        else:
            half = samples_per_batch // 2
            conds = ["control"] * half + ["treatment"] * (samples_per_batch - half)
        conditions.extend(conds)
        batches.extend([f"Batch_{b+1}"] * samples_per_batch)

    # Generate baseline expression means for each gene from a log-normal distribution.
    mu = np.random.lognormal(mean=4, sigma=1, size=n_genes)  # Shape: (n_genes,)

    # Determine the number of differentially expressed (DE) genes.
    n_de = int(de_fraction * n_genes)

    # Simulate batch effects for each batch.
    batch_effects = {}
    for b in range(n_batches):
        batch_label = f"Batch_{b+1}"
        batch_effects[batch_label] = np.random.lognormal(mean=0, sigma=batch_effect_sigma, size=n_genes)

    # Construct a matrix for batch effects with shape (n_genes, n_samples)
    batch_effect_matrix = np.column_stack([batch_effects[batch] for batch in batches])

    # Build condition effect matrix: for each sample, DE genes (first n_de) in treatment have increased expression.
    condition_effect_matrix = np.ones((n_genes, n_samples))
    for j, cond in enumerate(conditions):
        if cond == "treatment":
            condition_effect_matrix[:n_de, j] = fold_change

    # Compute the final mean for each gene in each sample.
    # mu[:, None] has shape (n_genes, 1) and broadcasts across all samples.
    final_mu = mu[:, None] * condition_effect_matrix * batch_effect_matrix

    # Negative binomial parameters: r is constant for all genes.
    r = 1.0 / dispersion
    # Compute success probability for each gene-sample combination.
    p = r / (r + final_mu)

    # Generate the count matrix in a vectorized fashion.
    counts = np.random.negative_binomial(r, p)

    # Create DataFrame with gene and sample identifiers.
    gene_ids = [f"Gene_{i+1}" for i in range(n_genes)]
    sample_ids = [f"Sample_{j+1}" for j in range(n_samples)]
    counts_df = pd.DataFrame(counts, index=gene_ids, columns=sample_ids)

    # Create a DataFrame for sample metadata.
    metadata_df = pd.DataFrame({
        "SampleID": sample_ids,
        "Condition": conditions,
        "Batch": batches
    })

    return counts_df, metadata_df

if __name__ == "__main__":
    # Generate the synthetic RNA-seq data with batch effects.
    counts_df, metadata_df = generate_synthetic_rnaseq_data(
        n_genes=10000,         # number of genes
        n_samples=12,          # total number of samples (e.g., 6 per batch if n_batches=2)
        n_batches=2,           # number of batches
        de_fraction=0.1,       # 10% of genes are differentially expressed
        fold_change=2.0,       # 2-fold upregulation in treatment for DE genes
        dispersion=0.2,        # dispersion parameter for count variability
        batch_effect_sigma=0.2,  # variability of batch effects
        seed=123               # seed for reproducibility
    )

    # Save the count matrix and metadata to CSV files.
    counts_df.to_csv("synthetic_bulk_rnaseq_counts_batches.csv")
    metadata_df.to_csv("sample_metadata.csv", index=False)
    print("Synthetic RNA-seq count data with batch effects saved to 'synthetic_bulk_rnaseq_counts_batches.csv'")
    print("Sample metadata saved to 'sample_metadata.csv'")

Synthetic RNA-seq count data with batch effects saved to 'synthetic_bulk_rnaseq_counts_batches.csv'
Sample metadata saved to 'sample_metadata.csv'


In [ ]:
# GPU-Accelerated version
def generate_synthetic_rnaseq_data_gpu(n_genes=10000, n_samples=12, n_batches=2, de_fraction=0.1,
                                       fold_change=2.0, dispersion=0.2, batch_effect_sigma=0.2, seed=123):
    cp.random.seed(seed)

    if n_samples % n_batches != 0:
        raise ValueError("n_samples must be divisible by n_batches")
    samples_per_batch = n_samples // n_batches

    conditions, batches = [], []
    for b in range(n_batches):
        if samples_per_batch % 2 == 0:
            conds = ["control"] * (samples_per_batch // 2) + ["treatment"] * (samples_per_batch // 2)
        else:
            half = samples_per_batch // 2
            conds = ["control"] * half + ["treatment"] * (samples_per_batch - half)
        conditions.extend(conds)
        batches.extend([f"Batch_{b+1}"] * samples_per_batch)

    # Generate baseline expression means on the GPU.
    mu = cp.random.lognormal(mean=4, sigma=1, size=n_genes)  # shape: (n_genes,)

    n_de = int(de_fraction * n_genes)

    # Simulate batch effects for each batch and stack them into a matrix.
    batch_effect_matrix = cp.column_stack([
        cp.random.lognormal(mean=0, sigma=batch_effect_sigma, size=n_genes) for _ in batches
    ])

    # Build the condition effect matrix.
    condition_effect_matrix = cp.ones((n_genes, n_samples))
    for j, cond in enumerate(conditions):
        if cond == "treatment":
            condition_effect_matrix[:n_de, j] = fold_change

    # Compute the final mean expression matrix.
    final_mu = mu[:, None] * condition_effect_matrix * batch_effect_matrix

    # Negative binomial parameters.
    r = 1.0 / dispersion
    p = r / (r + final_mu)

    # Generate counts on the GPU.
    counts = cp.random.negative_binomial(r, p)

    # Convert results to NumPy arrays for Pandas.
    counts_np = cp.asnumpy(counts)
    gene_ids = [f"Gene_{i+1}" for i in range(n_genes)]
    sample_ids = [f"Sample_{j+1}" for j in range(n_samples)]
    counts_df = pd.DataFrame(counts_np, index=gene_ids, columns=sample_ids)

    metadata_df = pd.DataFrame({
        "SampleID": sample_ids,
        "Condition": conditions,
        "Batch": batches
    })

    return counts_df, metadata_df

if __name__ == "__main__":
    counts_df, metadata_df = generate_synthetic_rnaseq_data_gpu(
        n_genes=10000,
        n_samples=12,
        n_batches=2,
        de_fraction=0.1,
        fold_change=2.0,
        dispersion=0.2,
        batch_effect_sigma=0.2,
        seed=123
    )

    counts_df.to_csv("synthetic_bulk_rnaseq_counts_batches_gpu.csv")
    metadata_df.to_csv("sample_metadata_gpu.csv", index=False)
    print("Synthetic RNA-seq count data with GPU acceleration saved to 'synthetic_bulk_rnaseq_counts_batches_gpu.csv'")
    print("Sample metadata saved to 'sample_metadata_gpu.csv'")

CUDARuntimeError: cudaErrorInsufficientDriver: CUDA driver version is insufficient for CUDA runtime version